## Import & global declaration

In [17]:
import collections
import numpy as np
import time
import os
from itertools import islice
import pandas as pd
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

# Local imports
from filters import *

# Global declarations
ALGORITHM = DSOR
INPUT_FOLDER = 'input'
OUTPUT_FOLDER = ALGORITHM.__name__
DATASET_FOLDERS = ['11','12','13','14','15','16','17','18','20','22','23','24','26','28','30','34','35','36','76']
MAX_LEN = 1 # max amount of frames to read from each sequence (set to none to run full set)

## Local declarations

In [18]:
# input function
def read_kitty(dir_names:list, return_labels:bool=True, max_len:int=None, input_folder:str='input'):
    """
    Opens a point cloud dataset following kitty formatting
    
    Parameters:
    dir_name: list of names of directorys containing a kitty sequence
    lables: if true, reads and returns labels as well
    max_len: maximum amount of point clouds to read from each directory
    input_folder: name of head folder containing the datasets
    
    Returns: Dictionaries for names, scans and optionally labels. Each with directory
    name as key and a list as value
    """

    names = {n:[] for n in dir_names}
    scans = {n:[] for n in dir_names}
    labels = {n:[] for n in dir_names}
    for dir_name in dir_names:
        # get datapaths
        scan_path = os.path.join(input_folder, dir_name, 'velodyne')
        scan_names = sorted([os.path.join(dp, f) 
                      for dp, dn, fn in os.walk(os.path.expanduser(scan_path)) 
                      for f in fn])

        label_path = os.path.join(input_folder, dir_name, 'labels')
        label_names = sorted([os.path.join(dp, f) 
                   for dp, dn, fn in os.walk(os.path.expanduser(label_path)) 
                   for f in fn])

        # assert all files have corresponding labels
        scan_path_len = len(scan_path)+1
        label_path_len = len(label_path)+1
        file_names = [scan[scan_path_len:-4] for scan in scan_names]
        assert all(name == label[label_path_len:-6] for name, label in zip(file_names, label_names)), f'file name missmatch in dir {dir_name}'
        
        names[dir_name] = file_names if not max_len else file_names[:max_len]

        # iterate datapaths
        iterator = zip(scan_names, label_names)
        iterator = iterator if not max_len else islice(iterator, max_len)
        for bin_file_name, label_file_name in iterator:

            # open and read pointcloud
            with open(bin_file_name, mode='rb') as file:
                scan = np.fromfile(file, dtype=np.float32)
                scan = np.reshape(scan, (-1,4))
                scans[dir_name] += [scan]

            # same for labels
            if return_labels:  
                with open(label_file_name, mode='rb') as file:
                    label = np.fromfile(file, dtype=np.int16)
                    label = np.reshape(label, (-1,2))
                    labels[dir_name] += [label]
                assert label.shape[0] == scan.shape[0] # assert 1 to 1 ratio

    if return_labels:
        return names, scans, labels 
    return names, scans

def write_one_kitty(dir_name:str, dataframes, labels=None, names=None):
    "writes numpy point cloud data as kitty format"
    output_path = os.path.join(dir_name)
    names = names if names else [str(i) for i in range(len(dataframes))]
    if not os.path.isdir(output_path):
        os.mkdir(output_path)
        os.mkdir(os.path.join(dir_name, 'velodyne'))
        os.mkdir(os.path.join(dir_name, 'labels'))
        
    # make velodyne data
    vel_dir = os.path.join(dir_name, 'velodyne')
    for data, name in zip(dataframes, names):
        data.tofile(f'{vel_dir}/{name}.bin', sep='', format='%s')
    
    # make label data
    if labels:
        lab_dir = os.path.join(dir_name, 'labels')
        for data, name in zip(labels, names):
            data.tofile(f'{lab_dir}/{name}.label', sep='', format='%s') 

# output function
def write_kitty(scans:dict, labels:dict=None, names:dict=None, folder:str='output'):
    """
    Writes a point cloud dataset following kitty formatting
    
    Parameters:
    scans: dictonary with directory name as key and a list of point clouds as value
    labels: optional dictonary with directory name as key and a list of labels as value
    names: dictonary with directory name as key and a list of file names as value
    folder: name of the top folder to put all output files and folders into
    """
    if not os.path.isdir(folder):
        os.mkdir(os.path.join(folder))
    keys = [*scans.keys()]
    for key in keys:
        kwargs = {
            'labels':None, 
            'names':None}
        if names:
            kwargs.update({'names':names[key]})
        if labels:
            kwargs.update({'labels':labels[key]})
        write_one_kitty(os.path.join(folder,key), scans[key], **kwargs)


def tuner(X, y, func, metric, params:dict, max_iter:int=10):
    """
    X: train data
    y: data labels
    func: algoritm function to call
    metric: evaluation metric to optimise
    param: dict with function params as key and as value: tuple of 
        (start value, increment function, decrement function)
    max_iter: loop will break after this many laps or if no update happens. 
        Can be set to None for exhaustive search (use with care)
    """
    metrics = {'metric':metric}
    met = lambda p: evaluate(X, y, func, metrics, target_label=110, **p)['metric'].mean()
    
    best = {key:val for key,(val,_,_) in params.items()}
    best = (met(best), best) # baseline
    tested = [best] # track tested params
    count=0 # track loop count
    
    def equal_dict(a,b):
        "checks if dicts contain equal values"
        return sorted(a.items()) == sorted(b.items())
        
    while True:
        old_best = best[1].copy()
        
        for param,(_,up,down) in params.items():
            _, b = best # unpack best params
            up_params, down_params = b.copy(), b.copy() # copy best
            up_params.update( {param:up(b[param])} ) # replace one param
            down_params.update( {param:down(b[param])} ) # replace one param
            results = [(met(p), p) for p in [down_params,up_params]]
            tested += results # add runs to log
            results += [best] # add baseline to compare
            new_best = max(results,key=lambda x:x[0]) # find new best
            best = new_best if new_best[0] > best[0] else best # only update on strictly greater than to avoid deadlock
            
        if equal_dict(old_best,best[1]): # if no update
            break
            
        count += 1
        if max_iter:
            if max_iter<=count:
                break
                
    return best, tested

def evaluate(points, labels, filter_func, metrics:dict, target_label:int=110, **func_kwargs): 
    import pandas as pd
    from itertools import chain
    
    flat_points = chain.from_iterable(points.values())
    flat_labels = chain.from_iterable(labels.values())
    
    mets=np.ndarray((0, len(metrics)+1) ,dtype=float)
    for X, y in zip(flat_points, flat_labels):
        outlier_labels = [target_label]
        start = time.time()
        mask = filter_func(X, **func_kwargs)
        m = [time.time()-start]
        m += [func(y, mask) for key, func in metrics.items()]
        mets = np.concatenate([mets,[m]])
    
    ind = [[key]*len(points[key]) for key in points.keys()]
    ind = chain.from_iterable(ind)
    return pd.DataFrame(mets,columns=['Time',*metrics.keys()],index=ind)

## Cleanup

In [19]:
def cleanup():
    names, points, labels = read_kitty(DATASET_FOLDERS, max_len=None, input_folder=INPUT_FOLDER)

    for folder, frames in points.items():
        l = labels[folder]
        assert len(l) == len(frames)
        for set_id, (label, frame) in enumerate(zip(l, frames)):
            P, index_map = np.unique(points[folder][set_id], return_index=True, axis=0)
            points[folder][set_id] = P
            labels[folder][set_id] = labels[folder][set_id][index_map]

    write_kitty(points, labels=labels, names=names, folder='clean')
    
# cleanup()

## Reading data

In [20]:
names, points, labels = read_kitty(DATASET_FOLDERS, max_len=MAX_LEN, input_folder=INPUT_FOLDER)
print('Point cloud shape:', points['11'][0].shape)
print(*points['11'][0][:5],sep='\n')
print()
print('Label shape:', labels['11'][0].shape)
print(*labels['11'][0][:5],sep='\n')

Point cloud shape: (115881, 4)
[-197.84776     26.662176     3.9865794   13.       ]
[-197.78802     25.835665     2.8061757    6.       ]
[-197.77983     25.658987     1.0338181   10.       ]
[-197.77368     26.125235     2.2110257    9.       ]
[-197.7581      26.23899      3.3880968   13.       ]

Label shape: (115881, 2)
[0 0]
[0 0]
[0 0]
[0 0]
[0 0]


In [21]:
outlier_labels = [110] # 110:label for snow in the air => outliers
binary_labels = {key:[np.isin(frame[:,0], outlier_labels) for frame in frames] for key, frames in labels.items()}
#binary_labels = np.isin(labels['11'][0][:,0], outlier_labels)

In [6]:
#write_kitty(points, labels, names, folder=OUTPUT_FOLDER) # No-op write to test function

## Filtering

In [40]:
# og params on https://bitbucket.org/autonomymtu/dsor_filter/src/master/launch/example.launch

increment_k = lambda k:k+2
decrement_k = lambda k:max(k-1,0)
increment_float = lambda f:f*2
decrement_float = lambda f:f*2/3

params = {
    'Sfactor':(0.001,increment_float,decrement_float),
    'r':(0.1,increment_float,decrement_float),
    'k':(4,increment_k,decrement_k),
}

alg = iterate_filter(DSOR, 2)
best, test_log = tuner(points, binary_labels, alg, f1_score, params, max_iter=2)

In [41]:
best,test_log

((0.8447144253595866, {'Sfactor': 0.001, 'r': 0.1, 'k': 4}),
 [(0.8447144253595866, {'Sfactor': 0.001, 'r': 0.1, 'k': 4}),
  (0.844655449277386, {'Sfactor': 0.0006666666666666666, 'r': 0.1, 'k': 4}),
  (0.8444879139304178, {'Sfactor': 0.002, 'r': 0.1, 'k': 4}),
  (0.798532613558412, {'Sfactor': 0.001, 'r': 0.06666666666666667, 'k': 4}),
  (0.40927462242076157, {'Sfactor': 0.001, 'r': 0.2, 'k': 4}),
  (0.8402296017184914, {'Sfactor': 0.001, 'r': 0.1, 'k': 3}),
  (0.84038588792533, {'Sfactor': 0.001, 'r': 0.1, 'k': 6})])

In [42]:
metrics = {
    'precision':precision_score, 
    'recall':recall_score, 
    'accuracy':accuracy_score,
    'f1':f1_score,
}
result = evaluate(points, binary_labels, alg, metrics, target_label=110, **best[1])

In [43]:
result.describe()

,Time,precision,recall,accuracy,f1
count,1.000000,1.00000,1.000000,1.000000,1.000000
mean,0.529797,0.84201,0.847436,0.961616,0.844714
std,NaN,NaN,NaN,NaN,NaN
min,0.529797,0.84201,0.847436,0.961616,0.844714
25%,0.529797,0.84201,0.847436,0.961616,0.844714
50%,0.529797,0.84201,0.847436,0.961616,0.844714
75%,0.529797,0.84201,0.847436,0.961616,0.844714
max,0.529797,0.84201,0.847436,0.961616,0.844714


## Anomaly detection

In [ ]:
frame = points['11'][0]
metrics = {
    'precision':precision_score, 
    'recall':recall_score, 
    'accuracy':accuracy_score
}

In [217]:
a_kernel = lambda f: 'poly'
another_kernel = lambda f:'sigmoid'
increment_percent = lambda f:(f+1.0)/2
decrement_percent = lambda f:(f+0.0)/2

params = {
    'kernel':('rbf',a_kernel,another_kernel),
    'contamination':(0.1,increment_percent,decrement_percent),
}

best, test_log = tuner(points, binary_labels, LOF, accuracy_score, params, max_iter=1)
print(best)
mask = OCS(frame[:,:3], contamination = 0.1)
[(key, func(binary_labels, mask)) for key, func in metrics.items()]

0.09999913704576247

In [232]:
params = {
    'k':(5,increment_k,decrement_k),
}

best, test_log = tuner(points, binary_labels, LOF, accuracy_score, params, max_iter=3)
print(best)
mask = LOF(frame[:,:3], metric='l1',contamination=0.1, **best)
[(key, func(binary_labels, mask)) for key, func in metrics.items()]

[('precision', 0.3072143596824301),
 ('recall', 0.24936957130848977),
 ('accuracy', 0.8382478577161053)]

In [224]:
mask = EE(frame[:,:3], contamination = 0.1)
[(key, func(binary_labels, mask)) for key, func in metrics.items()]

In [226]:
increment_int = lambda f:f*2
decrement_int = lambda f:f*2//3
increment_percent = lambda f:(f+1.0)/2
decrement_percent = lambda f:(f+0.0)/2

params = {
    'n_estimators':(100,increment_int,decrement_int),
    'max_features':(1.0,increment_percent,decrement_percent)
}

best, test_log = tuner(points, binary_labels, IF, accuracy_score, params, max_iter=3)
print(best)
mask = IF(frame[:,:3], contamination = 0.1, **best)
[(key, func(binary_labels, mask)) for key, func in metrics.items()]